In [ ]:
import kagglehub
import pandas as pd
import lux
from lux.vis.Vis import Vis

# KAGGLEHUB_CACHE 

# Download latest version
path = kagglehub.dataset_download("kalacheva/london-bike-share-usage-dataset")

print("Path to dataset files:", path)

df = pd.read_csv('./datasets/kalacheva/london-bike-share-usage-dataset/versions/1/LondonBikeJourneyAug2023.csv')
df

# Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)

Vis(["Start station=Kennington Lane Rail Bridge, Vauxhall","Bike number"],df)